In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

# Constants
G = 6.67430e-11  # Gravitational constant, m^3 kg^-1 s^-2
m1 = 5.972e24    # Mass of the first body (Earth), kg
m2 = 7.348e24    # Mass of the second body (Moon), kg
total_mass = m1 + m2
mu = m1 * m2 / total_mass  # Reduced mass

# Initial conditions
r1_initial = np.array([0, 0])
r2_initial = np.array([384400000, 0])  # Distance from Earth to Moon in meters
v1_initial = np.array([0, 0])
v2_initial = np.array([0, 1022])  # Orbital velocity of the Moon in m/s
y0 = np.concatenate((r1_initial, r2_initial, v1_initial, v2_initial))

# User modifiable variables
TotalElapsedTime = 120 * 24 * 3600  # Total elapsed time in seconds (100 days in this example)
AnimationDuration = 15  # Duration of the animation in seconds
SimulationTimeSteps = 3000  # Number of time steps for the simulation
Zoom_factor = 3  # Zoom factor for the plot ranges
x_range = np.array([-5e8, 5e8]) * Zoom_factor  # x-axis range for the plot
y_range = np.array([-5e8, 5e8]) * Zoom_factor  # y-axis range for the plot

# Define the system of equations for the two-body problem
def two_body(t, y):
    r1 = y[0:2]
    r2 = y[2:4]
    v1 = y[4:6]
    v2 = y[6:8]
    
    r = np.linalg.norm(r2 - r1)
    acc1 = G * m2 * (r2 - r1) / r**3
    acc2 = G * m1 * (r1 - r2) / r**3
    
    return np.concatenate((v1, v2, acc1, acc2))

# Time span
t_span = (0, TotalElapsedTime)
t_eval = np.linspace(t_span[0], t_span[1], SimulationTimeSteps)  # Fine time steps for simulation

# Solve the ODE
sol = solve_ivp(two_body, t_span, y0, t_eval=t_eval)

# Extract the solution
r1_sol = sol.y[0:2, :]
r2_sol = sol.y[2:4, :]

# Calculate center of mass and reduced mass position
com_sol = (m1 * r1_sol + m2 * r2_sol) / total_mass
r_rel = r2_sol - r1_sol
v_rel = sol.y[6:8, :] - sol.y[4:6, :]
r_rel_norm = np.linalg.norm(r_rel, axis=0)

# Angular momentum calculation from initial conditions
r_initial = r2_initial - r1_initial
v_initial = v2_initial - v1_initial
L = np.cross(r_initial, v_initial * mu)  # Angular momentum L = r x (m * v)
L_norm = np.linalg.norm(L)  # Angular momentum magnitude

# Define effective potential function
def effective_potential(r):
    return -G * m1 * m2 / r + (L_norm**2) / (2 * mu * r**2)

# Effective potential curve
r_min = np.min(r_rel_norm)
r_max = np.max(r_rel_norm)
r_vals = np.linspace(1e7, 2 * r_max, 1000)
v_eff = effective_potential(r_vals)

# Reduced mass position in the effective potential
r_eff_pos = r_rel_norm
v_eff_pos = effective_potential(r_eff_pos)

# Calculate y-axis range for effective potential plot
v_min = np.min([np.min(v_eff_pos),np.min(v_eff)])
v_max = np.max(v_eff_pos)
v_range = [v_min - 0.1 * np.abs(v_min), v_max + 0.1 * np.abs(v_max)]

# Number of frames for animation
num_frames = int(AnimationDuration * 25)  # 25 frames per second

# Indices for frames to be used in the animation
frame_indices = np.linspace(0, SimulationTimeSteps - 1, num_frames).astype(int)

# Create a subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=("Two-Body Problem", "Effective Potential"),
                    specs=[[{"type": "scatter"}, {"type": "scatter"}]])

# Initial trace for the two-body problem (Body 1 Trail)
fig.add_trace(go.Scatter(x=r1_sol[0, :1] - com_sol[0, 0], y=r1_sol[1, :1] - com_sol[1, 0], mode='lines', name='Body 1 Trail', 
                         line=dict(color='blue', width=1, dash='solid'), opacity=0.5), row=1, col=1)

# Initial trace for the two-body problem (Body 2 Trail)
fig.add_trace(go.Scatter(x=r2_sol[0, :1] - com_sol[0, 0], y=r2_sol[1, :1] - com_sol[1, 0], mode='lines', name='Body 2 Trail', 
                         line=dict(color='red', width=1, dash='solid'), opacity=0.5), row=1, col=1)

# Initial trace for the two-body problem (Body 1)
fig.add_trace(go.Scatter(x=r1_sol[0, :1] - com_sol[0, 0], y=r1_sol[1, :1] - com_sol[1, 0], mode='markers', name='Body 1', 
                         marker=dict(color='blue', size=8)), row=1, col=1)

# Initial trace for the two-body problem (Body 2)
fig.add_trace(go.Scatter(x=r2_sol[0, :1] - com_sol[0, 0], y=r2_sol[1, :1] - com_sol[1, 0], mode='markers', name='Body 2', 
                         marker=dict(color='red', size=8)), row=1, col=1)

# Initial trace for the effective potential
fig.add_trace(go.Scatter(x=r_vals, y=v_eff, mode='lines', name='Effective Potential'), row=1, col=2)

# Initial trace for the reduced mass in the effective potential
fig.add_trace(go.Scatter(x=[r_eff_pos[0]], y=[v_eff_pos[0]], mode='markers', name='Reduced Mass', 
                         marker=dict(color='green', size=10)), row=1, col=2)

# Animation frames
frames = []
for i in frame_indices:
    i = int(i)
    # frames.append(go.Frame(data=[
    #     go.Scatter(x=r1_sol[0, :i+1] - com_sol[0, :i+1], y=r1_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='blue', width=1), opacity=0.5, name='Body 1 Trail'),
    #     go.Scatter(x=r2_sol[0, :i+1] - com_sol[0, :i+1], y=r2_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='red', width=1), opacity=0.5, name='Body 2 Trail'),
    #     go.Scatter(x=[r1_sol[0, i] - com_sol[0, i]], y=[r1_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='blue', size=10), name='Body 1'),
    #     go.Scatter(x=[r2_sol[0, i] - com_sol[0, i]], y=[r2_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='red', size=10), name='Body 2'),
    #     go.Scatter(x=r_vals, y=v_eff, mode='lines', name='Effective Potential'),
    #     go.Scatter(x=[r_eff_pos[i]], y=[v_eff_pos[i]], mode='markers', name='Reduced Mass', marker=dict(color='green', size=10))
    # ]))

    frames.append(go.Frame(data=[
        go.Scatter(x=r1_sol[0, :i+1] - com_sol[0, :i+1], y=r1_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='blue', width=1), opacity=0.5, name='Body 1 Trail', xaxis='x1', yaxis='y1'),
        go.Scatter(x=r2_sol[0, :i+1] - com_sol[0, :i+1], y=r2_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='red', width=1), opacity=0.5, name='Body 2 Trail', xaxis='x1', yaxis='y1'),
        go.Scatter(x=[r1_sol[0, i] - com_sol[0, i]], y=[r1_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='blue', size=10), name='Body 1', xaxis='x1', yaxis='y1'),
        go.Scatter(x=[r2_sol[0, i] - com_sol[0, i]], y=[r2_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='red', size=10), name='Body 2', xaxis='x1', yaxis='y1'),
        go.Scatter(x=np.array([r1_sol[0, i] - com_sol[0, i], r2_sol[0, i] - com_sol[0, i]]), 
                   y=np.array([r1_sol[1, i] - com_sol[1, i], r2_sol[1, i] - com_sol[1, i]]), 
                   mode='lines', line=dict(color='green', width=2), name='Distance Line', xaxis='x1', yaxis='y1'),        
        go.Scatter(x=r_vals, y=v_eff, mode='lines', name='Effective Potential', xaxis='x2', yaxis='y2'),
        go.Scatter(x=[r_eff_pos[i]], y=[v_eff_pos[i]], mode='markers', name='Reduced Mass', marker=dict(color='green', size=10), xaxis='x2', yaxis='y2')
    ]))


# Update layout for animation
fig.update_layout(
    updatemenus=[dict(type='buttons', showactive=True,
                      buttons=[dict(label='Play', method='animate', args=[None, {"frame": {"duration": 1000 / 25, "redraw": True}, "fromcurrent": True}])])],
    xaxis=dict(range=x_range, autorange=False),
    yaxis=dict(range=y_range, autorange=False),
    xaxis2=dict(range=[1e7, 2 * r_max], autorange=False),
    yaxis2=dict(range=v_range, autorange=False),
    title="Two-Body Problem and Effective Potential Animation",
)

# Add frames to the figure
fig.frames = frames

fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

# Constants
G = 6.67430e-11  # Gravitational constant, m^3 kg^-1 s^-2
m1 = 5.972e24    # Mass of the first body (Earth), kg
m2 = 7.348e24    # Mass of the second body (Moon), kg
total_mass = m1 + m2
mu = m1 * m2 / total_mass  # Reduced mass

# Initial conditions
r1_initial = np.array([0, 0])
r2_initial = np.array([384400000, 0])  # Distance from Earth to Moon in meters
v1_initial = np.array([0, 0])
v2_initial = np.array([0, 1022])  # Orbital velocity of the Moon in m/s
y0 = np.concatenate((r1_initial, r2_initial, v1_initial, v2_initial))

# User modifiable variables
TotalElapsedTime = 200 * 24 * 3600  # Total elapsed time in seconds (100 days in this example)
AnimationDuration = 30  # Duration of the animation in seconds
SimulationTimeSteps = 1000  # Number of time steps for the simulation
Zoom_factor = 1  # Zoom factor for the plot ranges
x_range = np.array([-5e8, 5e8]) * Zoom_factor  # x-axis range for the plot
y_range = np.array([-5e8, 5e8]) * Zoom_factor  # y-axis range for the plot

# Define the system of equations for the two-body problem
def two_body(t, y):
    r1 = y[0:2]
    r2 = y[2:4]
    v1 = y[4:6]
    v2 = y[6:8]
    
    r = np.linalg.norm(r2 - r1)
    acc1 = G * m2 * (r2 - r1) / r**3
    acc2 = G * m1 * (r1 - r2) / r**3
    
    return np.concatenate((v1, v2, acc1, acc2))

# Time span
t_span = (0, TotalElapsedTime)
t_eval = np.linspace(t_span[0], t_span[1], SimulationTimeSteps)  # Fine time steps for simulation

# Solve the ODE
sol = solve_ivp(two_body, t_span, y0, t_eval=t_eval)

# Extract the solution
r1_sol = sol.y[0:2, :]
r2_sol = sol.y[2:4, :]

# Calculate center of mass and reduced mass position
com_sol = (m1 * r1_sol + m2 * r2_sol) / total_mass
r_rel = r2_sol - r1_sol
v_rel = sol.y[6:8, :] - sol.y[4:6, :]
r_rel_norm = np.linalg.norm(r_rel, axis=0)

# Angular momentum calculation from initial conditions
r_initial = r2_initial - r1_initial
v_initial = v2_initial - v1_initial
L = np.cross(r_initial, v_initial * mu)  # Angular momentum L = r x (m * v)
L_norm = np.linalg.norm(L)  # Angular momentum magnitude

# Define effective potential function
def effective_potential(r):
    return -G * m1 * m2 / r + (L_norm**2) / (2 * mu * r**2)

# Effective potential curve
r_min = np.min(r_rel_norm)
r_max = np.max(r_rel_norm)
r_vals = np.linspace(1e7, 2 * r_max, 1000)
v_eff = effective_potential(r_vals)

# Reduced mass position in the effective potential
r_eff_pos = r_rel_norm
v_eff_pos = effective_potential(r_eff_pos)

# Calculate y-axis range for effective potential plot
v_min = np.min([np.min(v_eff_pos),np.min(v_eff)])
v_max = np.max(v_eff_pos)
v_range = [v_min - 0.1 * np.abs(v_min), v_max + 0.1 * np.abs(v_max)]

# Number of frames for animation
num_frames = int(AnimationDuration * 25)  # 25 frames per second

# Indices for frames to be used in the animation
frame_indices = np.linspace(0, SimulationTimeSteps - 1, num_frames).astype(int)

# Create a subplot figure
fig = make_subplots(rows=1, cols=2, subplot_titles=("Two-Body Problem", "Effective Potential"),
                    specs=[[{"type": "scatter"}, {"type": "scatter"}]])

# Initial trace for the two-body problem (Body 1 Trail)
fig.add_trace(go.Scatter(x=r1_sol[0, :1] - com_sol[0, 0], y=r1_sol[1, :1] - com_sol[1, 0], mode='lines', name='Body 1 Trail', 
                         line=dict(color='blue', width=1, dash='solid'), opacity=0.5), row=1, col=1)

# Initial trace for the two-body problem (Body 2 Trail)
fig.add_trace(go.Scatter(x=r2_sol[0, :1] - com_sol[0, 0], y=r2_sol[1, :1] - com_sol[1, 0], mode='lines', name='Body 2 Trail', 
                         line=dict(color='red', width=1, dash='solid'), opacity=0.5), row=1, col=1)

# Initial trace for the two-body problem (Body 1)
fig.add_trace(go.Scatter(x=r1_sol[0, :1] - com_sol[0, 0], y=r1_sol[1, :1] - com_sol[1, 0], mode='markers', name='Body 1', 
                         marker=dict(color='blue', size=8)), row=1, col=1)

# Initial trace for the two-body problem (Body 2)
fig.add_trace(go.Scatter(x=r2_sol[0, :1] - com_sol[0, 0], y=r2_sol[1, :1] - com_sol[1, 0], mode='markers', name='Body 2', 
                         marker=dict(color='red', size=8)), row=1, col=1)

# Initial trace for the effective potential
fig.add_trace(go.Scatter(x=r_vals, y=v_eff, mode='lines', name='Effective Potential'), row=1, col=2)

# Initial trace for the reduced mass in the effective potential
fig.add_trace(go.Scatter(x=[r_eff_pos[0]], y=[v_eff_pos[0]], mode='markers', name='Reduced Mass', 
                         marker=dict(color='green', size=10)), row=1, col=2)

# Animation frames
frames = []
for i in frame_indices:
    i = int(i)
    # frames.append(go.Frame(data=[
    #     go.Scatter(x=r1_sol[0, :i+1] - com_sol[0, :i+1], y=r1_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='blue', width=1), opacity=0.5, name='Body 1 Trail'),
    #     go.Scatter(x=r2_sol[0, :i+1] - com_sol[0, :i+1], y=r2_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='red', width=1), opacity=0.5, name='Body 2 Trail'),
    #     go.Scatter(x=[r1_sol[0, i] - com_sol[0, i]], y=[r1_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='blue', size=10), name='Body 1'),
    #     go.Scatter(x=[r2_sol[0, i] - com_sol[0, i]], y=[r2_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='red', size=10), name='Body 2'),
    #     go.Scatter(x=r_vals, y=v_eff, mode='lines', name='Effective Potential'),
    #     go.Scatter(x=[r_eff_pos[i]], y=[v_eff_pos[i]], mode='markers', name='Reduced Mass', marker=dict(color='green', size=10))
    # ]))

    frames.append(go.Frame(data=[
        go.Scatter(x=r1_sol[0, :i+1] - com_sol[0, :i+1], y=r1_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='blue', width=1), opacity=0.5, name='Body 1 Trail', xaxis='x1', yaxis='y1'),
        go.Scatter(x=r2_sol[0, :i+1] - com_sol[0, :i+1], y=r2_sol[1, :i+1] - com_sol[1, :i+1], mode='lines', line=dict(color='red', width=1), opacity=0.5, name='Body 2 Trail', xaxis='x1', yaxis='y1'),
        go.Scatter(x=[r1_sol[0, i] - com_sol[0, i]], y=[r1_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='blue', size=10), name='Body 1', xaxis='x1', yaxis='y1'),
        go.Scatter(x=[r2_sol[0, i] - com_sol[0, i]], y=[r2_sol[1, i] - com_sol[1, i]], mode='markers', marker=dict(color='red', size=10), name='Body 2', xaxis='x1', yaxis='y1'),
        
        # go.Scatter(x=[r1_sol[0, i] - com_sol[0, i], r2_sol[0, i] - com_sol[0, i]], 
        #            y=[r1_sol[1, i] - com_sol[1, i], r2_sol[1, i] - com_sol[1, i]], 
        #            mode='lines', line=dict(color='green', width=2), name='Distance Line', xaxis='x1', yaxis='y1'),        
        
        go.Scatter(x=r_vals, y=v_eff, mode='lines', name='Effective Potential', xaxis='x2', yaxis='y2'),
        go.Scatter(x=[r_eff_pos[i]], y=[v_eff_pos[i]], mode='markers', name='Reduced Mass', marker=dict(color='green', size=10), xaxis='x2', yaxis='y2')
    ]))


# Update layout for animation
fig.update_layout(
    updatemenus=[dict(type='buttons', showactive=True,
                      buttons=[dict(label='Play', method='animate', args=[None, {"frame": {"duration": 1000 / 25, "redraw": True}, "fromcurrent": True}])])],
    xaxis=dict(range=x_range, autorange=False),
    yaxis=dict(range=y_range, autorange=False),
    xaxis2=dict(range=[1e7, 2 * r_max], autorange=False),
    yaxis2=dict(range=v_range, autorange=False),
    title="Two-Body Problem and Effective Potential Animation",
)

# Add frames to the figure
fig.frames = frames

fig.show()
